In [1]:
import rust_machine_learning
from syed_ml_lib import *
import os

In [2]:
# Setup the CNN
num_classifications = 10
max_pool_stride = 0
input_width = 256
input_height = 256
input_depth = 3
filters_per_conv_layer = [32]
filter_dimension = 3

CNN = rust_machine_learning.ConvolutionalNeuralNetwork(num_classifications, max_pool_stride, input_width, input_height, input_depth, filters_per_conv_layer, filter_dimension)

image_directory = "../data/animals-10/"
image_directory_path = os.path.abspath(image_directory)
CNN.set_image_loader(image_directory_path, input_width, input_height)

Reading images for classification "pecora"
Reading images for classification "cane"
Reading images for classification "ragno"
Reading images for classification "cavallo"
Reading images for classification "gallina"
Reading images for classification "scoiattolo"
Reading images for classification "farfalla"
Reading images for classification "gatto"
Reading images for classification "elefante"
Reading images for classification "mucca"


In [3]:
# Now train the model
learning_rate = 0.05
batch_size = 1
num_iter = 1


CNN.train_using_image_loader(learning_rate, batch_size, num_iter)

Time to load samples: 4
Finished preparing images
Finished feed forward in CNN Layers
Number of layers in network: 1
Layer shapes:
Shape of matrix is: 10 196608


GPUassert: invalid configuration argument cuda_kernels/src/cuda_kernels.cu 944
thread '<unnamed>' panicked at 'Matrices not compatible shape for mat mult! A: 10 196608 B: 2097152 1', matrix_lib/src/lib.rs:270:7
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: Matrices not compatible shape for mat mult! A: 10 196608 B: 2097152 1